In [9]:
import numpy as np
import random


# def transiction(s, a, p=-.5):
def transiction(s, a, p=2):
    # mu = np.sum(np.array(s) + np.array(a))/2 + p
    # return tuple(np.random.normal(mu, 1, 2))
    return np.array(s) + np.array(a) + np.random.normal(0, p, 2)
    # return random.gauss(s[0]+a, 1), random.gauss(s[1]+a, 1)

def generate(s, n):    
    for _ in range(n):
        a = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        # a = (1,0)
        yield s, a
        s = transiction(s, a)


s_0 = (1,1)
O = [(s, a) for (s, a) in generate(s_0, 100)]
O

[((1, 1), (-1, 0)),
 (array([ 0.17457422, -2.37913653]), (0, 1)),
 (array([-0.20934359, -2.77206563]), (0, -1)),
 (array([ 0.13895005, -5.4993873 ]), (1, 0)),
 (array([-1.78721487, -6.27173108]), (0, -1)),
 (array([-0.2328606 , -7.41651143]), (1, 0)),
 (array([-0.93852625, -9.16342832]), (1, 0)),
 (array([-4.05882011, -7.42535937]), (0, -1)),
 (array([-4.04140683, -7.17665938]), (0, -1)),
 (array([ -3.53340676, -10.93067656]), (0, 1)),
 (array([ 0.25196604, -6.00361381]), (-1, 0)),
 (array([-2.28595654, -5.95253708]), (1, 0)),
 (array([-3.40030998, -7.08058952]), (0, 1)),
 (array([-3.42863862, -5.40617324]), (0, -1)),
 (array([-5.13214457, -5.0103036 ]), (-1, 0)),
 (array([-5.29308977, -3.74358287]), (0, 1)),
 (array([-5.38084618,  0.32684079]), (1, 0)),
 (array([-2.23894698, -0.09108282]), (1, 0)),
 (array([-2.59569329,  0.46114801]), (-1, 0)),
 (array([-3.86693028,  2.27696239]), (-1, 0)),
 (array([-4.74271826,  2.77894336]), (-1, 0)),
 (array([-6.11026871,  5.11871743]), (1, 0)),
 (

$\cal{L}[\xi] = log \; p_0(s_0) \sum_{t=0}^{\mathrm{T}-1}log \; (\pi_{\theta}(a_t|s_t)p(s_{t+1}|s_t, a_t))$

In [10]:
from scipy.stats import norm

def ll(O, theta):
    p = lambda s_, s, a: norm.pdf(s_[0], (np.array(s[0]) + np.array(a[0])), theta) * norm.pdf(s_[1], (np.array(s[1]) + np.array(a[1])), theta)
    return np.sum([np.log(p(s, O[t-1][0], O[t-1][1])) for t,(s,_) in enumerate(O) if 0 < t])

# ll(O, -.5)
ll(O, 2)

-409.3760191007249

In [11]:
best_theta = 0
best_ll = -np.inf

for i in range(10):
    theta = 1 + i
    new_ll = ll(O, theta)
    if new_ll > best_ll:
        best_ll = new_ll
        best_theta = theta

print(best_theta, best_ll)

2 -409.3760191007249


In [12]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.FloatTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.FloatTensor)


p = Variable(torch.Tensor([2]), requires_grad=True)

loglike = torch.sum(normal.Normal(s[:,0] + a[:,0], p).log_prob(s_[:,0]) + normal.Normal(s[:,1] + a[:,1], p).log_prob(s_[:,1]))

loglike.backward()
gradiente = p.grad.clone()
print('like: ', loglike)
# gradiente.numpy()

like:  tensor(-409.3760, grad_fn=<SumBackward0>)


In [27]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.FloatTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.FloatTensor)

p = Variable(torch.randn(1), requires_grad=True)
while p.data.numpy()[0] <= 0:
    p = Variable(torch.randn(1), requires_grad=True)


learning_rate = 0.00002
e = np.inf
for t in range(10000):
    loglike = -torch.sum(normal.Normal(s[:,0] + a[:,0], p).log_prob(s_[:,0]) + normal.Normal(s[:,1] + a[:,1], p).log_prob(s_[:,1]))
    loglike.backward()
    
    new_p = p.data - learning_rate * p.grad.data
    if (new_p - p.data) == 0:
        break
    else:
        p.data = new_p
        p.grad.data.zero_()

print("loglike  =", loglike.data.numpy(), "p =", p.data.numpy())

loglike  = 408.95837 p = [1.9088361]
